In [7]:
import numpy as np

def repulsion_force(xi, yi, zi, xj, yj, zj, rv):
    """
    Calculate the repulsion force between two robots.

    Args:
    - xi, yi, zi: linear displacement of the first robot
    - xj, yj, zj: linear displacement of the second robot
    - rv: radius of the neighborhood sphere

    Returns:
    - fr_x, fr_y, fr_z: front, lateral, and vertical components of the repulsion force
    """
    distance = np.sqrt((xi - xj)**2 + (yi - yj)**2 + (zi - zj)**2)

    if distance < rv and distance != 0:
        direction_vector = np.array([xi - xj, yi - yj, zi - zj])
        unit_vector = direction_vector / np.linalg.norm(direction_vector)

        repulsion_force_magnitude = (rv - distance) **2 / distance
        repulsion_force_vector = repulsion_force_magnitude * unit_vector

        fr_x, fr_y, fr_z = repulsion_force_vector
    else:
        fr_x, fr_y, fr_z = 0.0, 0.0, 0.0

    return fr_x, fr_y, fr_z

In [14]:
def calculate_error(X, Xd):
    """
    Calculate the error vector.

    Args:
    - X: current position vector [xi, yi, zi]
    - Xd: target position vector [xd, yd, zd]

    Returns:
    - error vector [ex, ey, ez]
    """
    error = Xd - X
    return error

def sliding_surface(error, error_dot, lambda_val):
    """
    Calculate the sliding surface.

    Args:
    - error: error vector [ex, ey, ez]
    - error_dot: time derivative of the error vector [ex_dot, ey_dot, ez_dot]
    - lambda_val: sliding mode control parameter

    Returns:
    - sliding surface [sxi, syi, szi]
    """
    sliding_surface = lambda_val * error + error_dot
    return sliding_surface

def lyapunov_candidate(sliding_surface):
    """
    Calculate the Lyapunov candidate function.

    Args:
    - sliding_surface: sliding surface vector [sxi, syi, szi]

    Returns:
    - Lyapunov candidate function V
    """
    V = 0.5 * np.dot(sliding_surface, sliding_surface)
    return V

def sliding_mode_control(sliding_surface):
    """
    Implement sliding mode control.

    Args:
    - sliding_surface: sliding surface vector [sxi, syi, szi]

    Returns:
    - Control signals [u_x, u_y, u_z]
    """
    # You can adjust the control law as needed
    u_x = -sliding_surface[0]
    u_y = -sliding_surface[1]
    u_z = -sliding_surface[2]

    return u_x, u_y, u_z

def sliding_surface_derivative(error_dot, lambda_val, Xd_dot, X_dot):
    """
    Calculate the derivative of the sliding surface.

    Args:
    - error_dot: time derivative of the error vector [ex_dot, ey_dot, ez_dot]
    - lambda_val: sliding mode control parameter
    - Xd_dot: desired velocities of the target [xd_dot, yd_dot, zd_dot]
    - X_dot: current velocities [x_dot, y_dot, z_dot]

    Returns:
    - Derivative of the sliding surface [sxi_dot, syi_dot, szi_dot]
    """
    sliding_surface_dot = lambda_val * error_dot + (Xd_dot - X_dot)
    return sliding_surface_dot


def control_signal(sliding_surface, K1, K2):
    """
    Calculate the control signal to fulfill the sliding surface dynamics.

    Args:
    - sliding_surface: sliding surface vector [sxi, syi, szi]
    - K1, K2: positive constants

    Returns:
    - Control signals [u_x, u_y, u_z]
    """
    norm_s = np.linalg.norm(sliding_surface)
    control_signals = -K1 * sliding_surface - K2 * (sliding_surface / norm_s)  # Ensuring ||S|| is non-zero
    return control_signals


In [8]:
def aerial_robot_model_with_repulsion_and_sliding(xi, yi, zi, x_dot,
                                       y_dot, z_dot, u_x, u_y, u_z,
                                         f_d_x, f_d_y, f_d_z, a_bar,
                                           b_bar, delta_a, delta_b, neighbors, rv,
                                           Xd, resf, lambda_val):
    """
    A generic model for aerial robots with parameter uncertainties and repulsion forces.

    Args:
    - xi, yi, zi: linear displacement of the robot on each Cartesian axis
    - x_dot, y_dot, z_dot: time derivatives of xi, yi, zi
    - u_x, u_y, u_z: control signals for frontal, lateral, and vertical displacement
    - f_d_x, f_d_y, f_d_z: components of the coupled perturbation force
    - a_bar, b_bar: nominal values of parameters a and b
    - delta_a, delta_b: uncertainties in parameters a and b
    - neighbors: list of neighboring robots' positions as [xj, yj, zj]
    - rv: radius of the neighborhood sphere

    Returns:
    - x_ddot, y_ddot, z_ddot: time derivatives of x_dot, y_dot, z_dot
    """
    # Assuming a1i = a2i and b1i = b2i due to geometrical symmetry
    a1 = a_bar + np.random.uniform(-delta_a, delta_a)
    b1 = b_bar + np.random.uniform(-delta_b, delta_b)
    a2 = a1
    b2 = b1

    # Calculate time derivatives
    x_ddot = a1 * x_dot + b1 * (u_x + f_d_x)
    y_ddot = a2 * y_dot + b2 * (u_y + f_d_y)
    z_ddot = a_bar * z_dot + b_bar * (u_z + f_d_z)  # No uncertainty for z axis

    # Calculate repulsion forces
    for neighbor in neighbors:
        fr_x, fr_y, fr_z = repulsion_force(xi, yi, zi, *neighbor, rv)
        x_ddot += fr_x
        y_ddot += fr_y
        z_ddot += fr_z
    
    # Calculate error, sliding surface, and Lyapunov candidate
    X = np.array([xi, yi, zi])
    error = calculate_error(X, Xd)
    error_dot = -X_dot + Xd_dot #need to calculate them their value wont be given
    s_surface = sliding_surface(error, error_dot, lambda_val)

    # Apply sliding mode control
    u_x, u_y, u_z = sliding_mode_control(s_surface)

    s_surface_dot = sliding_surface_derivative(error_dot, lambda_val, Xd_dot, X_dot)

# Apply control signal to fulfill sliding surface dynamics
    control_signals = control_signal(s_surface, K1, K2)

    # Additional logic to move towards the target volume (Equation 5)
    if np.linalg.norm(X - Xd) < resf:
        # Adjust control signals to keep the robot inside the target volume
        u_x, u_y, u_z = 0.0, 0.0, 0.0

    return x_ddot, y_ddot, z_ddot, u_x, u_y, u_z

# Example usage:
# Replace the values with the specific parameters you have.
# a_bar, b_bar = 1.0, 2.0
# delta_a, delta_b = 0.1, 0.2
# neighbors = [[2.0, 3.0, 4.0], [5.0, 6.0, 7.0]]  # Replace with the positions of neighboring robots
# rv = 1.0  # Replace with the desired radius of the neighborhood sphere

# result = aerial_robot_model_with_repulsion(1.0, 2.0, 3.0,
#                                             0.1, 0.2, 0.3, 0.5,
#                                               0.2, 0.8, 0.1, 0.3,
#                                                 0.2, a_bar, b_bar, delta_a,
#                                                   delta_b, neighbors, rv)

# print("Result with repulsion forces:", result)